# Preparing and exploring data to apply supervised learning algorithm to highlight customer segmentation.

In [1]:
# Imported Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures

# custom imports
import prepare as p
import summarize as s

In [2]:
import acquire as a

df = a.acquire_data()

## Looking over raw data information
   * The datatypes need to be changed
   * The names should be pythonic and in lowercase

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## Quick view of the stats description for data.

In [4]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


# I'm going to evaluate missing values.

In [5]:
def missing_values(df):
    # calculate number of missing value for each attribute
    missing_counts = df.isna().sum()

    # calculate the percent of missing vals in each attribute
    total_rows = len(df)
    missing_percentages = (missing_counts / total_rows) * 100

    # create a summary df
    summary_df = pd.DataFrame({'Missing Values' : missing_counts, 'Percentage Missing (%)': missing_percentages})

    return summary_df

In [6]:
missing_values(df)

,Missing Values,Percentage Missing (%)
InvoiceNo,0,0.000000
StockCode,0,0.000000
Description,1454,0.268311
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
CustomerID,135080,24.926694
Country,0,0.000000


## Building prep function

In [7]:
def prepare_data(df):
    
    '''
    This function serves to accomplish cleaning of raw customir segmentation 
    data from kaggle.
    '''
    # import acquisition of raw data
    # df = a.acquire_data()

    # make feature names pythonic
    df.columns = [
        col.lower().replace(' ','_') for col in df.columns
    ]

    # renaming feature names for readability
    df.rename(columns={
        'invoiceno': 'invoice_no',
        'stockcode': 'stock_code',
        'invoicedate': 'invoice_date',
        'unitprice': 'unit_price',
        'customerid': 'customer_id'
    }, inplace=True)

    # Reassigning the invoicedate column to be a datetime type
    df.invoice_date = pd.to_datetime(df.invoice_date)
    
    # Sorting rows by the date and then set the index as that date
    # df = df.set_index("invoice_date").sort_index()
    
    df.drop(df[df["customer_id"].isnull()].index, axis=0, inplace=True)

    # Converting the following features to strings
    
    df['invoice_no'] = df['invoice_no'].astype(str)
    
    df['stock_code'] = df['stock_code'].astype(str)

    df['customer_id'] = df['customer_id'].astype(int)
    
    df['customer_id'] = df['customer_id'].astype(str)

    return df

In [8]:
prepped_df = prepare_data(df)

In [9]:
df.head(3)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom


In [10]:
s.summarize(prepped_df)

Shape of Data: 
(406829, 8)
Info: 
<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice_no    406829 non-null  object        
 1   stock_code    406829 non-null  object        
 2   description   406829 non-null  object        
 3   quantity      406829 non-null  int64         
 4   invoice_date  406829 non-null  datetime64[ns]
 5   unit_price    406829 non-null  float64       
 6   customer_id   406829 non-null  object        
 7   country       406829 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.9+ MB
None
Descriptions:
|              |   count | mean                          | min                 | 25%                 | 50%                 | 75%                 | max                 |      std |
|:-------------|--------:|:------------------------------|:----------

## Names are now pythonic, invoice_date is now set as index

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice_no    406829 non-null  object        
 1   stock_code    406829 non-null  object        
 2   description   406829 non-null  object        
 3   quantity      406829 non-null  int64         
 4   invoice_date  406829 non-null  datetime64[ns]
 5   unit_price    406829 non-null  float64       
 6   customer_id   406829 non-null  object        
 7   country       406829 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [12]:
df.describe()

,quantity,invoice_date,unit_price
count,406829.000000,406829,406829.000000
mean,12.061303,2011-07-10 16:30:57.879207424,3.460471
min,-80995.000000,2010-12-01 08:26:00,0.000000
25%,2.000000,2011-04-06 15:02:00,1.250000
50%,5.000000,2011-07-31 11:48:00,1.950000
75%,12.000000,2011-10-20 13:06:00,3.750000
max,80995.000000,2011-12-09 12:50:00,38970.000000
std,248.693370,NaN,69.315162


## Going to reevaluate the missing values in the dataset  

In [13]:
missing_values(df)

,Missing Values,Percentage Missing (%)
invoice_no,0,0.0
stock_code,0,0.0
description,0,0.0
quantity,0,0.0
invoice_date,0,0.0
unit_price,0,0.0
customer_id,0,0.0
country,0,0.0


* Again, customer_id and description seem to have many missing values
    * 0.27% missing data for decriptions.
    * 25.% missing data for customer_id. 

* could have used below to fill NaNs for descriptions

In [14]:
# # DataFrame with NaNs filled in the 'description' and 'customerid' column
# df = df.copy()
# df['customer_id'].fillna('Missing', inplace=True)

## Testing my function with code above.

In [15]:
import acquire as a
import prepare as p

df = a.acquire_data()

df = p.prepare_data(df)

df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,total_price,invoice_date_day,invoice_date_time,invoice_year,invoice_month,invoice_month_name,invoice_day,invoice_day_name,invoice_day_of_week,invoice_week_of_year,invoice_hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8


## Function is working appropriately

In [16]:
# yay no missing values! wooo!
p.missing_values(df)

,Missing Values,Percentage Missing (%)
invoice_no,0,0.0
stock_code,0,0.0
description,0,0.0
quantity,0,0.0
invoice_date,0,0.0
unit_price,0,0.0
customer_id,0,0.0
country,0,0.0
total_price,0,0.0
invoice_date_day,0,0.0


## Testing wrangle function

In [23]:
import wrangle as w

train, val, test, train_scaled, val_scaled, test_scaled, new_df, df_customers, df = w.wrangle_data()

In [24]:
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,total_price,invoice_date_day,invoice_date_time,invoice_year,invoice_month,invoice_month_name,invoice_day,invoice_day_name,invoice_day_of_week,invoice_week_of_year,invoice_hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,08:26:00,2010,12,December,1,Wednesday,2,48,8


* Wrangle function operational

In [25]:
p.missing_values(df)

,Missing Values,Percentage Missing (%)
invoice_no,0,0.0
stock_code,0,0.0
description,0,0.0
quantity,0,0.0
invoice_date,0,0.0
unit_price,0,0.0
customer_id,0,0.0
country,0,0.0
total_price,0,0.0
invoice_date_day,0,0.0


# That concludes my acquire and prep work now I need to explore the data.